# kzg-ezkl

Here's an example leveraging EZKL whereby the inputs to the model, and the model params themselves, are committed to using kzg-commitments inside a circuit.

In this setup:
- the commitments are publicly known to the prover and verifier


We leave the outputs of the model as public as well (known to the  verifier and prover). 


First we import the necessary dependencies and set up logging to be as informative as possible. 

In [1]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

from torch import nn
import ezkl
import os
import json
import logging

# uncomment for more descriptive logging 
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)


Now we define our model. It is a humble model with but a conv layer and a $ReLU$ non-linearity, but it is a model nonetheless

In [2]:
import torch
# Defines the model
# we got convs, we got relu, 
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5, stride=4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)

        return x


circuit = MyModel()

# this is where you'd train your model




We omit training for purposes of this demonstration. We've marked where training would happen in the cell above. 
Now we export the model to onnx and create a corresponding (randomly generated) input file.

You can replace the random `x` with real data if you so wish. 

In [3]:
x = torch.rand(1,*[3, 8, 8], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      "network.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open("input.json", 'w' ))

This is where the magic happens. We define our `PyRunArgs` objects which contains the visibility parameters for out model. 
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters 
- `output_visibility` defines the visibility of the model outputs

There are currently 6 visibility settings:
- `public`: known to both the verifier and prover (a subtle nuance is that this may not be the case for model parameters but until we have more rigorous theoretical results we don't want to make strong claims as to this). 
- `private`: known only to the prover
- `fixed`: known to the prover and verifier (as a commit), but not modifiable by the prover.
- `hashed`: the hash pre-image is known to the prover, the prover and verifier know the hash. The prover proves that the they know the pre-image to the hash. 
- `encrypted`: the non-encrypted element and the secret key used for decryption are known to the prover. The prover and the verifier know the encrypted element, the public key used to encrypt, and the hash of the decryption hey. The prover proves that they know the pre-image of the hashed decryption key and that this key can in fact decrypt the encrypted message.
- `polycommit`: unblinded advice column which generates a kzg commitment. This doesn't appear in the instances of the circuit and must instead be modified directly within the proof bytes.  

Here we create the following setup:
- `input_visibility`: "polycommit"
- `param_visibility`: "polycommit"
- `output_visibility`: public

We encourage you to play around with other setups :) 

Shoutouts: 

- [summa-solvency](https://github.com/summa-dev/summa-solvency) for their help with the poseidon hashing chip. 
- [timeofey](https://github.com/timoftime) for providing inspiration in our development of the el-gamal encryption circuit in Halo2. 

In [4]:
import ezkl

model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

data_path = os.path.join('input.json')

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "polycommit"
run_args.param_visibility = "polycommit"
run_args.output_visibility = "public"
run_args.variables = [("batch_size", 1)]





Now we generate a settings file. This file basically instantiates a bunch of parameters that determine their circuit shape, size etc... Because of the way we represent nonlinearities in the circuit (using Halo2's [lookup tables](https://zcash.github.io/halo2/design/proving-system/lookup.html)), it is often best to _calibrate_ this settings file as some data can fall out of range of these lookups.

You can pass a dataset for calibration that will be representative of real inputs you might find if and when you deploy the prover. Here we create a dummy calibration dataset for demonstration purposes. 

In [5]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

In [6]:
# generate a bunch of dummy calibration data
cal_data = {
    "input_data": [torch.cat((x, torch.rand(10, *[3, 8, 8]))).flatten().tolist()],
}

cal_path = os.path.join('val_data.json')
# save as json file
with open(cal_path, "w") as f:
    json.dump(cal_data, f)

res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

INFO ezkl.execute 2025-01-18 06:39:33,950 execute.rs:994 num calibration batches: 11
INFO tract_linalg.x86_64_fma 2025-01-18 06:39:33,953 x86_64_fma.rs:24 qmmm_i32: x86_64/avx2 activated
INFO tract_linalg.x86_64_fma 2025-01-18 06:39:33,955 x86_64_fma.rs:116 found f16c, added fake-f16 and q40-able kernels
INFO tract_linalg.x86_64_fma 2025-01-18 06:39:33,956 x86_64_fma.rs:119 mmm_f32, mmv_f32, sigmoid_f32, tanh_f32: x86_64/fma activated
WARNING ezkl.execute 2025-01-18 06:39:36,773 execute.rs:1295 

 <------------- Numerical Fidelity Report (input_scale: 13, param_scale: 13, scale_input_multiplier: 10) ------------->

+---------------+---------------+---------------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error    | median_error  | max_error     | min_error     | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | 

In [7]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

As we use Halo2 with KZG-commitments we need an SRS string from (preferably) a multi-party trusted setup ceremony. For an overview of the procedures for such a ceremony check out [this page](https://blog.ethereum.org/2023/01/16/announcing-kzg-ceremony). The `get_srs` command retrieves a correctly sized SRS given the calibrated settings file from [here](https://github.com/han0110/halo2-kzg-srs). 

These SRS were generated with [this](https://github.com/privacy-scaling-explorations/perpetualpowersoftau) ceremony. 

In [8]:
res = await ezkl.get_srs( settings_path)


INFO ezkl.execute 2025-01-18 06:39:43,426 execute.rs:697 SRS already exists at that path
INFO ezkl.execute 2025-01-18 06:39:43,441 execute.rs:596 read 4194564 bytes from file (vector of len = 4194564)
INFO ezkl.execute 2025-01-18 06:39:43,453 execute.rs:603 file hash: 90807800a1c3b248a452e1732c45ee5099f38b737356f5542c0584ec9c3ebb45


Here we setup verifying and proving keys for the circuit. As the name suggests the proving key is needed for ... proving and the verifying key is needed for ... verifying. 

In [9]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)




INFO ezkl.graph.model 2025-01-18 06:39:49,276 model.rs:1074 model layout...
INFO ezkl.circuit.table 2025-01-18 06:39:49,288 table.rs:433 Loading range check table from cache: "/home/thomas/.ezkl/cache/rangecheck_-1_1"
INFO ezkl.circuit.table 2025-01-18 06:39:49,291 table.rs:433 Loading range check table from cache: "/home/thomas/.ezkl/cache/rangecheck_0_16383"
INFO ezkl.pfsys 2025-01-18 06:39:50,368 mod.rs:542 VK took 1.267
INFO ezkl.graph.model 2025-01-18 06:39:50,401 model.rs:1074 model layout...
INFO ezkl.circuit.table 2025-01-18 06:39:50,403 table.rs:433 Loading range check table from cache: "/home/thomas/.ezkl/cache/rangecheck_-1_1"
INFO ezkl.circuit.table 2025-01-18 06:39:50,405 table.rs:433 Loading range check table from cache: "/home/thomas/.ezkl/cache/rangecheck_0_16383"
INFO ezkl.pfsys 2025-01-18 06:39:51,462 mod.rs:548 PK took 1.88
INFO ezkl.pfsys 2025-01-18 06:39:51,465 mod.rs:867 done saving verification key ✅
INFO ezkl.pfsys 2025-01-18 06:39:52,736 mod.rs:850 done saving 

We now need to generate the (partial) circuit witness. These are the model outputs (and any hashes) that are generated when feeding the previously generated `input.json` through the circuit / model. 

In [10]:
!export RUST_BACKTRACE=1

witness_path = "witness.json"

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path, vk_path)


INFO ezkl.pfsys 2025-01-18 06:40:17,095 mod.rs:810 loaded verification key ✅


As a sanity check you can "mock prove" (i.e check that all the constraints of the circuit match without generate a full proof). 

In [11]:

res = ezkl.mock(witness_path, compiled_model_path)

INFO ezkl.execute 2025-01-18 06:40:18,648 execute.rs:1346 Mock proof
INFO ezkl.graph.model 2025-01-18 06:40:18,700 model.rs:1074 model layout...
INFO ezkl.circuit.table 2025-01-18 06:40:18,703 table.rs:433 Loading range check table from cache: "/home/thomas/.ezkl/cache/rangecheck_-1_1"
INFO ezkl.circuit.table 2025-01-18 06:40:18,704 table.rs:433 Loading range check table from cache: "/home/thomas/.ezkl/cache/rangecheck_0_16383"


Now we generate a full proof. 

In [12]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

INFO ezkl.pfsys 2025-01-18 06:40:25,892 mod.rs:833 loaded proving key ✅
INFO ezkl.pfsys 2025-01-18 06:40:25,909 mod.rs:604 proof started...
INFO ezkl.graph.model 2025-01-18 06:40:25,996 model.rs:1074 model layout...
INFO ezkl.circuit.table 2025-01-18 06:40:25,999 table.rs:433 Loading range check table from cache: "/home/thomas/.ezkl/cache/rangecheck_-1_1"
INFO ezkl.circuit.table 2025-01-18 06:40:26,000 table.rs:433 Loading range check table from cache: "/home/thomas/.ezkl/cache/rangecheck_0_16383"
INFO ezkl.pfsys 2025-01-18 06:40:28,009 mod.rs:643 proof took 2.83


{'instances': [['2a58140100000000000000000000000000000000000000000000000000000000']], 'proof': '0x23cc9b225ca04dbfb574c4ff4b359cb72907d8a2d4c3bb0419869fb417965eb928c1f044e116e17e4f78444aa963295ddbbe05a28b096dbf846e27a6c437c45c0bba36481aff0bc4e2562c9fa8f7082a52549f0f5ee12286c50f890a0acbe6e6242ff2c9c1260a06bda0920ca5480da0f19465273fc1f735e4f8c0733cb56ca12a4e46e71cdcdf0b16cdad22085c3c98575670a6ff0ac41a8d397efaa5d1c1e30dc8bfd24c6bfb1e557b60bbbf403458bad09abee72cf8bb760c882f61a864bc2abe82a0bdb1b27121abe0961114b36cf6f02a2eec9ebb9552db065d4d2c3d3a09c5961fbd78f62b2d27abd1523fe059a605434b823b2e1a53f335debfb3601f10ae920b376fc73db310645eb17524ccfe11f53636e7164fbe7c226f4b4d355a0e20cb35bef99a312e9badd10edaab0cdb48127114437a64112cd83eb5d01fe4149d1215532fe3ff52f443cc38ec13874e7bcff3b14c418c3342488481bee2502e13d201e79e741b1f63e098fd9437682af6b9fae2f55392633c561a5629acf721027b2ea66f7a16c4e91adbdd3d82ea556af1740d05e4043e05c075142f5b9002591f96dd97129a9693729163ecb2a7b23c59bb1e6ea1855febb1f38bbd63c601824d

Now we need to swap out the public commitments inside the corresponding proof bytes

In [13]:
res = ezkl.swap_proof_commitments(proof_path, witness_path)

And verify it as a sanity check. 

In [14]:
# VERIFY IT

res 


res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")

INFO ezkl.pfsys 2025-01-18 06:41:30,820 mod.rs:810 loaded verification key ✅
INFO ezkl.execute 2025-01-18 06:41:30,842 execute.rs:2415 verify took 0.14
INFO ezkl.execute 2025-01-18 06:41:30,844 execute.rs:2420 verified: true


verified


We can now create an EVM / `.sol` verifier that can be deployed on chain to verify submitted proofs using a view function.

In [ ]:

abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = await ezkl.create_evm_verifier(
        vk_path,
        
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True


## Verify on the evm

In [ ]:
# Make sure anvil is running locally first
# run with $ anvil -p 3030
# we use the default anvil node here
import json

address_path = os.path.join("address.json")

res = await ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

with open(address_path, 'r') as file:
    addr = file.read().rstrip()

In [ ]:
# make sure anvil is running locally
# $ anvil -p 3030

res = await ezkl.verify_evm(
    addr,
    proof_path,
    "http://127.0.0.1:3030"
)
assert res == True